In [3]:

#start of every round, initialize the player
class Player:
    def __init__(self, health, currHealth, gold, experience, weapon, armor, skills, elementalResistances):
        self.health = health + armor.health
        self.gold = gold
        self.weapon = weapon
        self.armor = armor
        self.experience = experience
        self.skillList = skills 
        self.criticalRate = 0.05 + weapon.criticalRate;
        self.attackDamage = 10 + weapon.attackDamage
        self.magicDamage = 10 + weapon.magicDamage
        self.physicalDefense = 10 + armor.physicalDefense
        self.magicDefense = 10 + armor.magicDefense
        self.luck = 0.05 + armor.luck
        self.elementalResistances = {'Fire':0+armor.elementalResistances['Fire'], 
                                     'Water':0+armor.elementalResistances['Water'], 
                                     'Electric':0+armor.elementalResistances['Electric'],
                                     'Ice':0+armor.elementalResistances['Ice'],
                                     'Dark':0+armor.elementalResistances['Dark'],
                                     'Light':0+armor.elementalResistances['Light'],
                                     'Physical':0+armor.elementalResistances['Physical']
                                    }
    
    def adjustHealth(self, health): #Health can be positive or negative 
        self.health += health
    
    def changeWeapon(self, weapon): 
        self.weapon = weapon
    
    def changeArmor(self, armor):
        self.armor = armor
    
    def adjustattackDamage(self, value):
        self.attackDamage += value

    def adjustmagicDamage(self, value):
        self.magicDamage += value
    
    def printElementalResistances(self):
        for y in self.elementalResistances[x]:
            print (y,':',self.elementalResistances[x][y])
    
    def getWeapon(self):
        return self.weapon
    
    def getArmor(self):
        return self.armor
        
class Weapon:
    def __init__(self, name, attackDamage, magicDamage, criticalRate, element):
        self.name = name
        self.attackDamage = attackDamage 
        self.magicDamage = magicDamage 
        self.criticalRate = criticalRate
        self.element = element
        
    def getName(self):
        return self.name

class Armor:
    def __init__(self, name, health, physicalDefense, magicDefense, luck, elementalResistances):
        self.name = name
        self.physicalDefense = physicalDefense
        self.health = health
        self.magicDefense = magicDefense 
        self.luck = luck 
        self.elementalResistances = elementalResistances

    def getName(self):
        return self.name

class Enemy:
    def __init__(self, health, skillList, dropsDictionary, name, goldReward, experienceReward, attackDamage, magicDamage, 
                 physicalDefense, magicDefense, elementalResistances):
        #elemental resistances could be a dictionary with elements and respective resistances 
        self.health = health
        self.skillList = skillList
        self.dropsDictionary = dropsDictionary
        self.name = name
        self.goldReward = goldReward
        self.experienceReward = experienceReward
        self.attackDamage = attackDamage
        self.magicDamage = magicDamage
        self.physicalDefense = physicalDefense
        self.magicDefense = magicDefense 
        self.elementalResistances = elementalResistances
    
    def adjustHealth(self, health):
        self.health += health
    
    def getName(self):
        return self.name


In [5]:
from random import randint
import random
from IPython.display import clear_output
class Skill:
    def __init__(self, name, baseDmg, variationDmg, scope, energyCost, element, skillType):
        self.name = name
        self.baseDmg = baseDmg
        self.variationDmg = variationDmg 
        self.scope = scope
        self.energyCost = energyCost
        self.element = element 
        self.skillType = skillType
    
    def getName(self):
        return self.name
    
    def getScope(self):
        return self.scope
    
    def executeSkill(self, target, user): #skill type refers to physical, magical, or neutral damage 
        rawDamage = randint(self.baseDmg-self.variationDmg, self.baseDmg+self.variationDmg) #establish lower and upper bounds of damage 
        if (self.skillType == 'Physical'):
            overallDamage = rawDamage + user.attackDamage - target.physicalDefense - target.elementalResistances[self.element]
            if (random.uniform(0, 1) <= user.criticalRate):
                overallDamage *= 1.5
                print("Critical Strike!")
            if (target.elementalResistances[self.element] > 5):
                print("Target is resistant to this element!")
            if (target.elementalResistances[self.element] < 0):
                print("Target is weak to this element!")
            if (overallDamage < 0):
                overallDamage = 0 #this is to make sure the damage does not go negative before the next line 
            target.adjustHealth(-1 * overallDamage)
            if (isinstance(user, Player)): #adjust print statement based on if the user was the player or the enemy
                print("You dealt " + str(overallDamage) + " damage to the enemy " + target.name + "!" )
            if (isinstance(user, Enemy)):
                print("The enemy dealt " + str(overallDamage) + " damage to you!" )                
            
        if (self.skillType == 'Magical'):
            overallDamage = rawDamage + user.magicDamage - target.magicDefense - target.elementalResistances[self.element]
            if (target.elementalResistances[self.element] > 5):
                print("Target is resistant to this element!")
            if (target.elementalResistances[self.element] < 0):
                print("Target is weak to this element!")
            if (overallDamage < 0):
                overallDamage = 0 #this is to make sure the damage does not go negative before the next line
            target.adjustHealth(-1 * overallDamage)
            if (isinstance(user, Player)): #adjust print statement based on if the user was the player or the enemy
                print("You dealt " + str(overallDamage) + " damage to the enemy " + target.name + "!" )
            if (isinstance(user, Enemy)):
                print("The enemy dealt " + str(overallDamage) + " damage to you!" )   
            ##put a function here that checks the health for game overs and/or enemy defeats 
            #or just check after we call this function 
        #clear_output()
        return overallDamage 
    
class BuffDebuffSkill:
    def __init__(self, name, stat, adjustment, skillType):
        self.name = name
        self.stat = stat #tracks which stat in particular will be adjusted 
        self.skillType = skillType #tracks how much the stat gets adjusted 
        if (self.skillType == "Buff"):
            self.adjustment = adjustment 
        else:
            self.adjustment = adjustment * -1 
    
    def executeSkill(self, target, user):
        if (self.stat == "Attack Damage"):
            target.adjustattackDamage(self.adjustment)
        if (self.stat == "Magic Damage"):
            target.adjustmagicDamage(self.adjustment)  

    def getName(self):
        return self.name
        
            
        
        

In [2]:
import json
with open('unlockables.json') as json_file:
    data = json.load(json_file)
#read in the unlockables and store them in this list that 
#can be accessed everywhere
allUnlocksFromJson = data
unlockedSkills_string = allUnlocksFromJson["skillUnlocks"]
unlockedSkills_string

['Fireball', 'Freezing Bolt', 'Motivate', 'Zap', 'Dark Strike']

In [4]:
# Skill(Name, base dmg, var dmg, energy cost, element, skillType)
fireballSpell = Skill("Fireball", 7.5, 2.5, "Single", 8, 'Fire', "Magical")
darkStrike = Skill("Dark Strike", 7.5, 2.5, "Single", 8, 'Dark', "Magical")
freezeSpell = Skill("Freezing Bolt", 7.5, 2.5, "Single", 8, 'Ice', "Magical")
motivate = BuffDebuffSkill("Motivate", "Attack Damage", 5, "Buff")
zap = Skill("Zap", 7.5, 2.5, "Single", 8, "Electric", "Magical")

# Weapon (Name, Attack, Magic, Crit Rate, Element)
# Armor (Name, Health, physical defense, magical defense, luck, elemental resist)
scythe = Weapon("Scythe", 3, 3, 0.05, 'Physical')
silverSword = Weapon("Silver Sword", 5, 0, 0.03, 'Physical')
plating = Armor("Plating", 15, 1, 1, 1, {'Fire':3, 'Water': 2, 'Electric': 0, 'Ice': 5, 'Dark': 3, 'Light': -1, 'Physical': 0})

In [8]:
######################################################################
#
# All code in this section is for the preparation screen, where 
# the player can equip skills, armor, weapons, etc.
#
######################################################################
import json
from IPython.display import clear_output

    
#let's say they pass in 'Fireball', we want it to return fireballSpell 
#we could have a list of skills and we check each name to see if it 
#matches the parameter! 

#The user enters in the string representation, the program
#reads from the JSON that has the skill String Representations,
#and then it should blurt out the Skill version of the string representation. 

#this list should have every attainable spell and attack 
everySkill = [fireballSpell, freezeSpell, motivate, zap] 
everyWeapon = [scythe, silverSword] 
everyArmor = [plating] 




#this function loops through the
#everySkill list and appends the skill to the
#given list if the skill is found in everySkill
def findSkill (specificSkillName, skillList):
    for skill in everySkill:
        if (skill.getName() == specificSkillName):
            skillList.append(skill)

def findWeapon (specificWeaponName, weaponList):
    for weapon in everyWeapon:
        if (weapon.getName() == specificWeaponName):
            weaponList.append(weapon)            

def findArmor (specificArmorName, armorList):
    for armor in everyArmor:
        if (armor.getName() == specificArmorName):
            armorList.append(armor)  
            

def skillEquality (skillName, skill2):
    return skillName == skill2.getName()
        
def printList(list):
    for item in list:
        print (item)

#read through the json file and keep track
#of all the skills that are unlocked. 
with open('unlockables.json') as json_file:
    data = json.load(json_file)
#read in the unlockables and store them in this list that 
#can be accessed everywhere
allUnlocksFromJson = data
unlockedSkills_string = allUnlocksFromJson["skillUnlocks"]
unlockedWeapons_string = allUnlocksFromJson["weaponUnlocks"]
unlockedArmors_string = allUnlocksFromJson["armorUnlocks"]

#THIS SHOULD BE MOVED TO START OF PROGRAM AFTER
with open('saveData.json') as json_file:
    data = json.load(json_file)
allSaveData = data
equippedSkills_string = allSaveData["equippedSkills"]
equippedSkills = [] 
for eachSkill in equippedSkills_string:
    #for every string representation of a skill in the equippedSkills_string list,
    #find the corresponding skill from the everySkill list and add it to equippedSkills
    findSkill(eachSkill, equippedSkills)
#equips the current weapon from the save file 
for weapon in everyWeapon:
    if (weapon.getName().upper() == allSaveData["currentWeaponEquip"].upper()):
        currentWeapon = weapon
#equips the current armor from the save file 
for armor in everyArmor:
    if (armor.getName().upper() == allSaveData["currentArmorEquip"].upper()):
        currentArmor = armor
#synchronize the inventories
weaponInventory = []
armorInventory = []
for weapon in allSaveData["weaponInventory"]:
    findWeapon(weapon, weaponInventory)
for armor in allSaveData["armorInventory"]:
    findArmor(armor, armorInventory)

protagonist = Player(500, 400, 500, 4, currentWeapon, currentArmor, equippedSkills, {'Fire':3, 'Water': 2, 'Electric': 0, 'Ice': 5, 'Dark': 3, 'Light': -1, 'Physical': 0})
    
    
    
keepEquipping = True
while (keepEquipping):
    firstEquipPrompt = input("What would you like to equip?\nType in 'weapons', 'armor', 'skills' to equip those respectively.\nYou may also type in 'cancel' to stop equipping.")
    if (firstEquipPrompt.upper() == "CANCEL"):
        keepEquipping = False
    
    if (firstEquipPrompt.upper() == "ARMOR"):
        #
        # CODE BELOW FOR EQUIPPING ARMOR
        #
        clear_output()
        keepArmoring = True
        while (keepArmoring):
            invalidEquipArmor = False
            print("You currently have " + protagonist.getArmor().getName() + " equipped.")
            armorInput = input("Which armor would you like to equip?\nType in 'inventory' to see all the armor you have.\nType in 'cancel' to stop equipping weapons.")
            
            if (armorInput.upper() == "INVENTORY"):
                clear_output()
                print("Here are the armor in your inventory:")
                for armor in armorInventory:
                    print (armor.getName())
                    invalidEquipArmor = True
                    
            if (armorInput.upper() == "CANCEL"):
                clear_output()
                keepArmoring = False
                invalidEquipArmor = True
                
            if (protagonist.getArmor().getName().upper() == armorInput.upper()):
                clear_output()
                print("You already have that armor equipped.\n")
                invalidEquipArmor = True
                
            if (invalidEquipArmor == False):
                for armor in armorInventory:
                    if (armor.getName().upper() == armorInput.upper()):
                        protagonist.changeArmor(armor)
                        clear_output()
                        print("Successfully equipped " + protagonist.getArmor().getName() + "!")
                        keepArmoring = False
                if (keepArmoring):
                    clear_output()
                    print("Invalid selection.\n")

        with open('saveData.json', 'w') as outfile:
            allSaveData["currentArmorEquip"] = protagonist.getArmor().getName()
            json.dump(allSaveData, outfile) 
        #
        # CODE FOR EQUIPPING ARMOR ENDS HERE
        #            
    
    if (firstEquipPrompt.upper() == "WEAPONS"):
        #
        # CODE BELOW FOR EQUIPPING WEAPONS
        #
        clear_output()
        keepWeaponing = True 
        while (keepWeaponing):
            invalidEquipWeapon = False
            print("You currently have " + protagonist.getWeapon().getName()  + " equipped.")
            weaponInput = input("Which weapon would you like to equip?\nType in 'inventory' to see all the weapons you have.\nType in 'cancel' to stop equipping weapons.")
            
            if (weaponInput.upper() == "INVENTORY"):
                clear_output()
                print("Here are the weapons in your inventory:")
                for weapon in weaponInventory:
                    print (weapon.getName())
                    invalidEquipWeapon = True
                    
            if (weaponInput.upper() == "CANCEL"):
                clear_output()
                keepWeaponing = False
                invalidEquipWeapon = True
                
            if (protagonist.getWeapon().getName().upper() == weaponInput.upper()):
                clear_output()
                print("You already have that weapon equipped.\n")
                invalidEquipWeapon = True
                
            if (invalidEquipWeapon == False):
                for weapon in weaponInventory:
                    if (weapon.getName().upper() == weaponInput.upper()):
                        protagonist.changeWeapon(weapon)
                        clear_output()
                        print("Successfully equipped " + protagonist.getWeapon().getName() + "!")
                        keepWeaponing = False
                if (keepWeaponing):
                    clear_output()
                    print("Invalid selection.\n")

        with open('saveData.json', 'w') as outfile:
            allSaveData["currentWeaponEquip"] = protagonist.getWeapon().getName()
            json.dump(allSaveData, outfile) 
        #
        # CODE FOR EQUIPPING WEAPONS ENDS HERE
        #
    if (firstEquipPrompt.upper() == "SKILLS"):
        #
        # CODE BELOW FOR EQUIPPING SKILLS 
        #
        clear_output()
        keepPrompting = True
        while(keepPrompting): #keep asking the user to equip a skill until its valid
            #otherOptions will track if the user picks one of the other options (equips or unlocks)
            #checkInvalidSkill will check if the user enters in a valid skill input
            #continueEquipping will be used later to track if the user can keep equipping skills 
            otherOptions = False
            checkInvalidSkill = True
            continueEquipping = True
            userInput = input("Which skill would you like to equip?\nType in 'equips' to see what you currently have equipped and 'unlocks' to see all the skills you have unlocked.\nYou can also type in 'cancel' to stop equipping skills.")
            clear_output()
            if (userInput.upper() == "EQUIPS"):
                print("Here are the skills you currently have equipped:\n")
                printList(equippedSkills_string)
                otherOptions = True
            if (userInput.upper() == "UNLOCKS"):
                print("Here are the skills you currently have unlocked:\n")
                printList(unlockedSkills_string)  
                otherOptions = True
            if (userInput.upper() == "CANCEL"):
                keepPrompting = False   
                checkInvalidSkill = False
            if (otherOptions == False):
                #this for loop checks if the skill input the user submitted is an unlocked skill
                for skill_string in unlockedSkills_string:
                        if (userInput.upper() == skill_string.upper()):
                            #this for loop makes sure the skill the user inputted is not CURRENTLY equippedt
                            for skillsEquipped in equippedSkills_string:
                                if (userInput.upper() == skillsEquipped.upper()):
                                    print("You already have " + userInput + " equipped!")
                                    continueEquipping = False
                            #If the skill is not already equipped, then proceed with the equip process
                            if (continueEquipping):
                                clear_output()
                                print("Here are the skills you currently have equipped:\n")
                                #print every skill you have equipped currently 
                                printList(equippedSkills_string)
                                replaceInput = input("Which skill would you like to replace with " + skill_string + "?\nYou can also type in 'none' to cancel this selection.\n")
                                if (replaceInput.upper() == 'NONE'):
                                    checkInvalidSkill = False
                                    clear_output()
                                else:
                                    #replace the skill at the index where we find it 
                                    for index, skill3 in enumerate(equippedSkills_string):
                                        if (skill3.upper() == replaceInput.upper()):
                                            equippedSkills_string[index] = skill_string
                                            clear_output()
                                            print(userInput + " was successfully equipped!")
                                            checkInvalidSkill = False   
                                            continueEquipping = input("Would you like to continue equipping skills? Type in 'yes' to continue equipping skills.")
                                            if (continueEquipping.upper() == "YES"):
                                                keepPrompting = True
                                                clear_output()
                                            else:
                                                keepPrompting = False
                                        
                if (checkInvalidSkill):
                    print(userInput + " is not a valid selection.")

        #equippedSkills is the same as equippedSkills_string,
        #except the strings are turned into their Skill representations 

        for eachSkill in equippedSkills_string:
            #for every string representation of a skill in the equippedSkills_string list,
            #find the corresponding skill from the everySkill list and add it to equippedSkills
            findSkill(eachSkill, equippedSkills)
        with open('saveData.json', 'w') as outfile:
            allSaveData["equippedSkills"] = equippedSkills_string
            json.dump(allSaveData, outfile) 
        #
        # Code for equipping skills ends here 
        #
######################################################################
#
# Code for the preparation screen ends here 
# NOTE: The dumping into the JSON could and should be moved off the if statements
######################################################################

#should track progress on a stage by stage basis
#and save data after every stage
#maybe include a retry option at the end of every stage
#that just loads the save data 

What would you like to equip?
Type in 'weapons', 'armor', 'skills' to equip those respectively.
You may also type in 'cancel' to stop equipping.cancel


In [21]:
allUnlocksFromJson
armorInventory
weaponInventory

[<__main__.Weapon at 0x287f24cb6a0>, <__main__.Weapon at 0x287f24cba90>]

In [6]:
import json
#unlockables = {}
#unlockables['skillUnlocks'] = [] ##so one entry in the unlockables dictionary is a list of skills that are unlocked 
#unlockables['skillUnlocks'].append('Fireball')
#unlockables['skillUnlocks'].append('Freezing Bolt')
#unlockables['skillUnlocks'].append('Motivate')
#unlockables['skillUnlocks'].append('Electric')

#this would probably be at the end of the program
#where we dump all the progress into the JSON file. 

#so the previous cell would take in the unlockables from the JSON
#and this line below would send it back into the JSON after we've updated
#it with new unlockables
with open('unlockables.json', 'w') as outfile:
    json.dump(allUnlocksFromJson, outfile)

#once we develop the program more,
#we can just have a custom starting unlockables.json
#that the player starts with.

#for now, this cell should create/modify the unlockables

#at the end of the program, the json file should probably be modified
#with any new unlocks. 

In [ ]:
def skillBattle(enemyTroops): #enemy troops should be a list of the enemies you're fighting
    #make sure each enemy has a number just in case of dupes
    #while adding, check for dupes 
    #clear_output()
    chooseSkill = input("Which skill would you like to use?\n")
    printFlag1 = True
    for eachSkill in equippedSkills:
        if (chooseSkill.upper() == eachSkill.getName().upper()):
            if (eachSkill.getScope() == "Single"):
                print("Here are the current enemies you are fighting:\n")
                for enemy in enemyTroops:
                    print(enemy.getName())
                target = input("Which enemy would you like to target?")
                validEnemy = False
                for enemy in enemyTroops:
                    if (target.upper() == enemy.getName()):
                        eachSkill.executeSkill(enemy, protagonist)
                        validEnemy = True
                    if (validEnemy == False):
                        clear_output()
                        print("Not a valid target.")
        if (printFlag1):
            print(userInput + " is not an unlocked skill you can use.")

In [6]:
baseDmg = 3
variationDmg = 1
randint(baseDmg-variationDmg, baseDmg+variationDmg)

2

In [8]:
elementResistance = {'Fire':3, 'Water': 2, 'Electric': 0, 'Ice': 5, 'Dark': 3, 'Light': -1, 'Physical': 0}
scythe = Weapon(3, 3, 0.01, 'Physical')
plating = Armor(15, 1, 1, 1, elementResistance)
mainCharacter = Player(100, 0, 0, scythe, plating, 0, elementResistance)
slimeEnemy = Enemy(50, 'Slime', [freezeSpell, motivate], {plating: 0.25}, 10, 10, 5, 5, 3, 3, elementResistance)
fireballSpell = Skill('FIREBALL', 7.5, 2.5, 8, 'Fire', 'Magical')
attack = Skill('ATTACK', 3,1,0,'Physical', 'Physical')

In [10]:
attack.executeSkill(slimeEnemy, mainCharacter)

Critical Strike!
You dealt 21.0 damage to the enemy Slime!


21.0

In [5]:
elementResistance = {'Fire':3, 'Water': 2} ##example for element resistance 

In [6]:
elementResistance['Fire']

3